# Model 1

In [39]:
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

neu = '0'
pos = '0'
neg = '1'
title='aif'

f = open('/home/ydw/capston/python/data/sum(fromPH.D+twitter)/text.txt', 'r', encoding='utf-8')
text = f.read().splitlines()
f.close()    
f = open('/home/ydw/capston/python/data/sum(fromPH.D+twitter)/result.txt', 'r', encoding='utf-8')
r_y = f.read().splitlines()
f.close()

y = []
for i in r_y:
    if(i == '2'):
        y.append(neu)
    elif(i == '0'):
        y.append(pos)
    else:
        y.append(neg)
        
f = open('/home/ydw/capston/python/data/twitter/positive.txt', 'r', encoding='utf-8')
p_text = f.read().splitlines()
f.close()    

f = open('/home/ydw/capston/python/data/twitter/negative.txt', 'r', encoding='utf-8')
n_text = f.read().splitlines()
f.close()    

p_text = p_text[35000:35000+3000]
n_text = n_text[35000:35000+3000]

p_text = [' '.join(re.sub("(RT )|(@\S+)|(\w+:\/\/\S+)|", "", doc).split()) for doc in p_text]
n_text = [' '.join(re.sub("(RT )|(@\S+)|(\w+:\/\/\S+)|", "", doc).split()) for doc in n_text]

for i in p_text:
    text.append(i)
    y.append(pos)
    
for i in n_text:
    text.append(i)
    y.append(neg)

#One-hot encoding Model
text_train, text_test, y_train, y_test = train_test_split(text, y, test_size=0.2, random_state=1)
pipe_b = make_pipeline(CountVectorizer(ngram_range=(1,3), min_df=0), LogisticRegression())
pipe_b.fit(text_train, y_train)

f = open('/home/ydw/capston/python/data/application/reply_{}_100.txt'.format(title), 'r', encoding='utf-8')
text = f.read().splitlines()
f.close()    

f = open('/home/ydw/capston/python/data/application/{}.txt'.format(title), 'r', encoding='utf-8')
r_y = f.read().splitlines()
f.close()    

y = []
for i in r_y:
    if(i == 'l'):
        y.append(neu)
    elif(i == 'p'):
        y.append(pos)
    elif(i == 'n'):
        y.append(neg)
    else:
        y.append('NA')
        
pre_y = []
for doc in text:
    rdoc = [doc]
    pre_y.append(pipe_b.predict(rdoc)[0])
    
count = 0
for i in range(len(y)):
    if(y[i] == pre_y[i]):
        count = count+1
        
a = pre_y
count/len(y), pre_y.count('0'), y.count('0')

(0.68, 82, 74)

# Model 2

In [40]:
import re
import numpy as np
import fasttext

neu = '0'
pos = '1'
neg = '0'
title='aif'

f = open('/home/ydw/capston/python/data/expert/neutral.txt', 'r', encoding='utf-8')
text = f.read().splitlines()
f.close()    

y = []
for i in range(len(text)):
    y.append(neu)

f = open('/home/ydw/capston/python/data/twitter/positive.txt', 'r', encoding='utf-8')
p_text = f.read().splitlines()
f.close()    

f = open('/home/ydw/capston/python/data/twitter/negative.txt', 'r', encoding='utf-8')
n_text = f.read().splitlines()
f.close()    

p_text = p_text[35000:35000+3000]
n_text = n_text[35000:35000+3000]

p_text = [' '.join(re.sub("(RT )|(@\S+)|(\w+:\/\/\S+)|", "", doc).split()) for doc in p_text]
n_text = [' '.join(re.sub("(RT )|(@\S+)|(\w+:\/\/\S+)|", "", doc).split()) for doc in n_text]

for i in p_text:
    text.append(i)
    y.append(pos)
    
for i in n_text:
    text.append(i)
    y.append(neg)

#fasttext Model
for i in range(len(text)):
    text[i] = '__label__'+y[i]+' '+text[i]+'\n'
    
fasttext_train = open("/home/ydw/capston/python/data/sum(fromPH.D+twitter)/fasttext_data/fasttext_train.txt", "w", encoding='utf-8')
fasttext_test = open("/home/ydw/capston/python/data/sum(fromPH.D+twitter)/fasttext_data/fasttext_test.txt", "w", encoding='utf-8')
for i in text:
    if(np.random.uniform() < 0.2):
        fasttext_test.write(i)
    else:
        fasttext_train.write(i)
        
classifier = fasttext.supervised('/home/ydw/capston/python/data/sum(fromPH.D+twitter)/fasttext_data/fasttext_train.txt',
                                 '/home/ydw/capston/python/data/sum(fromPH.D+twitter)/fasttext_data/classifier',
                                 dim=20, loss='softmax', epoch=10, label_prefix='__label__')

f = open('/home/ydw/capston/python/data/application/reply_{}_100.txt'.format(title), 'r', encoding='utf-8')
text = f.read().splitlines()
f.close()    

f = open('/home/ydw/capston/python/data/application/{}.txt'.format(title), 'r', encoding='utf-8')
r_y = f.read().splitlines()
f.close()    

y = []
for i in r_y:
    if(i == 'l'):
        y.append(neu)
    elif(i == 'p'):
        y.append(pos)
    elif(i == 'n'):
        y.append(neg)
    else:
        y.append('NA')
        
pre_y = []
for doc in text:
    rdoc = [doc]
    pre_y.append(classifier.predict(rdoc)[0][0])

count = 0
for i in range(len(y)):
    if(y[i] == pre_y[i]):
        count = count+1
        
b = pre_y
count/len(y), pre_y.count('0'), y.count('0')

(0.83, 91, 86)

# Cross

In [41]:
index = []
re_text = []
for i in range(len(a)):
    if(a[i]=='1' and b[i]=='1'):
        index.append(i)
        re_text.append(text[i])
        a[i] = '3'
    elif(a[i]=='0' and b[i]=='1'):
        a[i] = '0'
    elif(a[i]=='0' and b[i]=='0'):
        a[i] = '2'        

In [42]:
f = open('/home/ydw/capston/python/data/twitter/positive.txt', 'r', encoding='utf-8')
p_text = f.read().splitlines()
f.close()    

f = open('/home/ydw/capston/python/data/twitter/negative.txt', 'r', encoding='utf-8')
n_text = f.read().splitlines()
f.close()    

p_text = p_text[:20000]
n_text = n_text[:20000]

p_text = [' '.join(re.sub("(RT )|(@\S+)|(\w+:\/\/\S+)|", "", doc).split()) for doc in p_text]
n_text = [' '.join(re.sub("(RT )|(@\S+)|(\w+:\/\/\S+)|", "", doc).split()) for doc in n_text]

text = []
y = []

for i in p_text:
    text.append(i)
    y.append('0')
    
for i in n_text:
    text.append(i)
    y.append('1')

text_train, text_test, y_train, y_test = train_test_split(text, y, test_size=0.2, random_state=1)
pipe_n = make_pipeline(CountVectorizer(ngram_range=(1,3), min_df=0), LogisticRegression())
pipe_n.fit(text_train, y_train)

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=0,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
  ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [43]:
pre_re_y = []
for doc in re_text:
    rdoc = [doc]
    pre_re_y.append(pipe_n.predict(rdoc)[0])
    
k = 0
for i in index:
    a[i] = pre_re_y[k]
    k=k+1

# Test

In [44]:
f = open('/home/ydw/capston/python/data/application/{}.txt'.format(title), 'r', encoding='utf-8')
r_y = f.read().splitlines()
f.close()    

y = []
for i in r_y:
    if(i == 'l'):
        y.append('2')
    elif(i == 'p'):
        y.append('0')
    elif(i == 'n'):
        y.append('1')
    else:
        y.append('NA')
        
count = 0
for i in range(len(y)):
    if(y[i] == a[i]):
        count = count+1
print(count/len(y))

y.count('0'), a.count('0'),y.count('1'), a.count('1'),y.count('2'), a.count('2')

0.53


(12, 9, 24, 18, 62, 73)